In [1]:
import json
import numpy as np
import matplotlib.pyplot as plt
from scipy import signal
import queue
import time
from concurrent.futures import ThreadPoolExecutor, Future
import threading

In [2]:
import dispatcher as DP
import controller as CT
import cell_search as CS
import mib_decoding as MD
import channel_estimate as CE
import sib1_decoding as SD

In [3]:
# read metadata
meta_filename = '../data/LTE_downlink_806MHz_2022-04-09_30720ksps.sigmf-meta'

with open(meta_filename, 'r') as f:
    meta_data = json.load(f)

In [4]:
data_type = meta_data['global']['core:datatype']
Fs = meta_data['global']['core:sample_rate']

if data_type == 'cf32_le':
    dtype = np.complex64
elif data_type == 'ci16_le':
    dtype = np.int16
elif data_type == 'ci8_le':
    dtype = np.int8
else:
    raise ValueError(f"Unsupported data type: {data_type}")

In [5]:
# read data
data_filename = '../data/LTE_downlink_806MHz_2022-04-09_30720ksps.sigmf-data'

with open(data_filename, 'rb') as f:
    tmp = np.fromfile(f,dtype=dtype)  # Adjust dtype as needed

In [6]:
scale = 2**(-15)
rx = scale*tmp[0::2] + scale*1j*tmp[1::2]

In [7]:
fc = 10e6
rx_unfr = rx*np.exp(-2j*np.pi*fc*np.arange(len(rx))/Fs)

h = signal.remez(60, [0, 4.5e6, 5.5e6, Fs/2], [1, 0], fs=Fs)
w, H = signal.freqz(h, fs=Fs)
rxf = np.convolve(rx_unfr,h)

In [8]:
rxf = rxf[::2]
Fs = Fs // 2

# system parameters
N_FFT = Fs // 15000
N_CP = 144 // 2 # normal CP
N_CP_extra = 16 // 2
half_frame = (Fs*5) // 1000 # 5 ms 

In [9]:
class ActorSystem:
    def __init__(self, num_workers=3):
        """
        Initializes the ActorSystem with a set number of worker threads.

        Args:
            num_workers (int): The number of threads in the thread pool for parallel task processing.
        """
        self.executor = ThreadPoolExecutor(max_workers=num_workers)
        self.actors = {}  # Dictionary to store actors by their names
        self.task_queue = queue.Queue()  # Central task queue for actors awaiting processing
        self.actor_locks = {}  # Dictionary to manage locks for each actor
        self.start_workers()  # Initialize worker threads

    def create_actor(self, actor):
        """
        Registers an actor within the ActorSystem.

        Args:
            actor: The actor instance to be registered.

        Returns:
            The registered actor instance.
        """
        name = actor.name
        self.actors[name] = actor
        self.actor_locks[name] = threading.Lock()  # Assign a lock to control access to this actor
        return actor

    def enqueue_task(self, actor):
        """
        Adds an actor to the task queue for processing.

        Args:
            actor: The actor instance to enqueue.
        """
        self.task_queue.put(actor)

    def process_tasks(self):
        """
        Continuously retrieves and processes tasks from the task queue.
        """
        while True:
            actor = self.task_queue.get()  # Retrieve the next actor to process
            if actor:
                actor_lock = self.actor_locks[actor.name]
                # Submit the actor task to the executor, ensuring it has exclusive access
                self.executor.submit(self.run_actor_task, actor, actor_lock)
            
            # Break the loop if an actor signals to stop processing
            if actor.stop_processing:
                break

    def run_actor_task(self, actor, actor_lock):
        """
        Executes the actor's task while holding a lock to ensure exclusive access.

        Args:
            actor: The actor instance whose task is being executed.
            actor_lock: The threading lock assigned to this actor.
        """
        with actor_lock:  # Lock to ensure only one thread processes this actor at a time
            actor()  # Execute the actor's task

    def start_workers(self):
        """
        Starts worker threads to continuously process tasks from the task queue.
        """
        for _ in range(self.executor._max_workers):
            task_thread = threading.Thread(target=self.process_tasks, daemon=True)
            task_thread.start()

    def send_message(self, target_actor_name, message):
        """
        Sends a message to a specific actor and enqueues it for processing.

        Args:
            target_actor_name (str): The name of the target actor.
            message: The message to be sent to the actor.
        """
        target_actor = self.actors.get(target_actor_name)
        if target_actor:
            target_actor.store_message(message)  # Add the message to the actor's queue
            self.enqueue_task(target_actor)  # Enqueue the actor for processing

    def stop(self):
        """
        Stops the ActorSystem by signaling all actors to stop and shutting down the thread pool.
        """
        for actor in self.actors.values():
            actor.stop()  # Signal each actor to stop processing
        self.executor.shutdown(wait=True)  # Gracefully shut down the thread pool, waiting for all threads to complete


In [10]:
# Initialize Actor System with 8 workers
actor_system = ActorSystem(num_workers=8)

# Define buffer length and state transition list
buffer_len = Fs // 10  # Circular buffer size, adjusted for half-frame data storage
state_can = ['CellSearch', 'MIBDecode', 'SIB1Decode']  # State transitions for Controller

# Create actors and register them with the ActorSystem
# CellSearch Actor: Performs initial cell search for synchronization and signal detection
actor_system.create_actor(CS.CellSearch('CellSearch', actor_system, N_FFT, N_CP, Fs))

# Dispatcher Actor: Manages circular buffer and distributes data to different actors
actor_system.create_actor(DP.Dispatcher('Dispatcher', actor_system, buffer_len, N_FFT, N_CP, N_CP_extra))

# Controller Actor: Manages system states and controls data flow between actors
actor_system.create_actor(CT.Controller('Controller', actor_system, state_can, N_FFT, N_CP, N_CP_extra, Fs))

# ChannelEstimate Actor: Responsible for channel estimation required for decoding
actor_system.create_actor(CE.ChannelEstimate('ChannelEstimate', actor_system, N_CP, N_CP_extra, Fs))

# MIBDecode Actor: Decodes the Master Information Block (MIB) for system information
actor_system.create_actor(MD.MIBDecode('MIBDecode', actor_system))

# SIB1Decode Actor: Decodes the System Information Block 1 (SIB1) for additional cell info
actor_system.create_actor(SD.SIB1Decode('SIB1Decode', actor_system))

# Define window parameters for data processing
win_len = N_FFT  # Window length for each data chunk
win = N_FFT      # Initial position of the data window
ob_len = Fs // 2     # Total observable length of data

# Continuous message sending to Dispatcher actor
while win < ob_len:
    # Slice data chunk from received signal for this window
    data = rxf[win - win_len : win]
    data_info = {
        'type': 'W',             # Type: Data
        'destination': 'Dispatcher',  # Target Actor
        'data': data             # Data payload
    }
    
    # Send data to the Dispatcher actor
    actor_system.send_message('Dispatcher', data_info)
    
    # Move the window forward by win_len
    win += win_len
    
    # Pause to send every data in 50 ms
    time.sleep(0.05)

# Allow time for all tasks to complete (optional based on system requirements)
# time.sleep(20)

# Stop the Actor System (optional, depends on specific application needs)
# actor_system.stop()


Initial Cell Searching...
Initial Cell Searching...
Initial Cell Searching...
Initial Cell Searching...
Initial Cell Searching...
Initial Cell Searching...
Initial Cell Searching...
Initial Cell Searching...
Initial Cell Searching...
Initial Cell Searching...
Initial Cell Searching...
Initial Cell Searching...
Initial Cell Searching...
Initial Cell Searching...
Initial Cell Searching...
Initial Cell Searching...
Initial Cell Searching...
Initial Cell Searching...
Initial Cell Searching...
Initial Cell Searching...
Initial Cell Searching...
Initial Cell Searching...
Initial Cell Searching...
Initial Cell Searching...
Initial Cell Searching...
Initial Cell Searching...
Initial Cell Searching...
Initial Cell Searching...
Initial Cell Searching...
Initial Cell Searching...
Initial Cell Searching...
Initial Cell Searching...
Initial Cell Searching...
Initial Cell Searching...
Initial Cell Searching...
Initial Cell Searching...
Initial Cell Searching...
Initial Cell Searching...
Initial Cell

KeyboardInterrupt: 

Processing Requeued Message ...
Processing Requeued Message ...
